In [1]:
'''
The code is used to train BC imitator, or pretrained GAIL imitator
'''

import argparse
import tempfile
import os.path as osp
import gym
import logging
from tqdm import tqdm

import tensorflow as tf

from baselines.gail import mlp_policy
from baselines import bench
from baselines import logger
from baselines.common import set_global_seeds, tf_util as U
from baselines.common.misc_util import boolean_flag

#from baselines.gail.run_mujoco import runner
from baselines.gail.dataset.mujoco_dset import Mujoco_Dset 

##from baselines.common.mpi_adam import MpiAdam

In [2]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment
network = "gfootball_impala_cnn"
scenario = "../_scenarios/academy/pass_n_shoot.scenic"
load_path = "/home/ubuntu/ScenicGFootBall/training/gfrl/_saved_models/pass_shoot_5M"



In [3]:


import tensorflow.compat.v1 as tf
import multiprocessing

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])



ncpu = multiprocessing.cpu_count()
config = tf.ConfigProto(allow_soft_placement=True,
                        intra_op_parallelism_threads=ncpu,
                        inter_op_parallelism_threads=ncpu)
config.gpu_options.allow_growth = True
tf.Session(config=config).__enter__()

from gfrl.common.mybase import ppo2
model = ppo2.learn(
    network=network,
    total_timesteps=0,
    env=vec_env,
    nsteps=512,
    load_path=load_path
    )




/home/ubuntu/.local/lib/python3.7/site-packages/scenic/core/errors.py:160: UserWarning: unable to install sys.excepthook to format Scenic backtraces
  warnings.warn('unable to install sys.excepthook to format Scenic backtraces')


Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
pygame 2.0.1 (SDL 2.0.14, Python 3.7.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
Logging to /tmp/openai-2021-04-26-22-33-37-225230





Instructions for updating:
Use `tf.cast` instead.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.





Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where






In [4]:
from gfrl.dm.my_runner import MyTrajRunner

from baselines.common.vec_env.dummy_vec_env import DummyVecEnv
vec_env = DummyVecEnv([lambda _i=i: \
                        create_single_scenic_environment(_i,scenario) for i in
                        range(1)])

runner = MyTrajRunner(env=vec_env, model=model, nsteps=512, num_episodes=5)
mb_obs, mb_rewards, mb_dones, mb_actions, mb_values, mb_neglogpacs, mb_states, epinfos, mb_gt = runner.run()



Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic
in run


In [5]:
print(mb_obs.shape, mb_actions.shape)

(150, 72, 96, 16) (150,)


In [6]:
mb_actions.reshape(-1,1).shape

(150, 1)

In [7]:
expert_file_name = "../_data/pns_50.npz"
import numpy as np 
np.savez(expert_file_name, obs=mb_obs, acs=mb_actions)

In [8]:
td = np.load(expert_file_name)

In [9]:
list(td.keys())

['obs', 'acs']

In [10]:
class Dset(object):
    def __init__(self, inputs, labels, randomize):
        self.inputs = inputs
        self.labels = labels
        assert len(self.inputs) == len(self.labels)
        self.randomize = randomize
        self.num_pairs = len(inputs)
        self.init_pointer()

    def init_pointer(self):
        self.pointer = 0
        if self.randomize:
            idx = np.arange(self.num_pairs)
            np.random.shuffle(idx)
            self.inputs = self.inputs[idx, :]
            self.labels = self.labels[idx, :]

    def get_next_batch(self, batch_size):
        # if batch_size is negative -> return all
        if batch_size < 0:
            return self.inputs, self.labels
        if self.pointer + batch_size >= self.num_pairs:
            self.init_pointer()
        end = self.pointer + batch_size
        inputs = self.inputs[self.pointer:end, :]
        labels = self.labels[self.pointer:end, :]
        self.pointer = end
        return inputs, labels

In [11]:
import numpy as np
class GFDset(object):
    
    def __init__(self, expert_path, randomize=True):
        
        traj_data = np.load(expert_path)
        self.inputs = traj_data["obs"]
        self.labels = traj_data["acs"].reshape(-1,1)
        assert len(self.inputs) == len(self.labels)
        self.randomize = randomize
        self.num_pairs = self.inputs.shape[0]
        self.init_pointer()

    def init_pointer(self):
        self.pointer = 0
        if self.randomize:
            idx = np.arange(self.num_pairs)
            np.random.shuffle(idx)
            self.inputs = self.inputs[idx, :]
            self.labels = self.labels[idx, :]

    def get_next_batch(self, batch_size):
        # if batch_size is negative -> return all
        if batch_size < 0:
            return self.inputs, self.labels
        if self.pointer + batch_size >= self.num_pairs:
            self.init_pointer()
        end = self.pointer + batch_size
        inputs = self.inputs[self.pointer:end, :]
        labels = self.labels[self.pointer:end, :]
        self.pointer = end
        return inputs, labels

In [12]:
from gfrl.base.run_my_ppo2 import create_single_scenic_environment
    
scenario = "../_scenarios/academy/pass_n_shoot.scenic"
env = create_single_scenic_environment(0,scenario)

from baselines.common.policies import build_policy
network = "gfootball_impala_cnn"
#Check Network parameters
network_kwargs = {}
policy_fn = build_policy(env, network, **network_kwargs)

Scenic Environment:  ../_scenarios/academy/pass_n_shoot.scenic


In [13]:

policy_with_value = policy_fn()

In [14]:
policy_with_value.sess

In [15]:
pi = policy_with_value

In [16]:
pi._evaluate([pi.action, pi.vf, pi.state, pi.neglogp], observation=td['obs'][0])

FailedPreconditionError: Attempting to use uninitialized value pi/convnet/residual_3_0/conv_2d/w
	 [[node pi/convnet/residual_3_0/conv_2d/w/read (defined at /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'pi/convnet/residual_3_0/conv_2d/w/read':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/.local/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/home/ubuntu/.local/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in sta

In [17]:
pi.step(td['obs'][0])

FailedPreconditionError: Attempting to use uninitialized value pi/linear/b
	 [[node pi/linear/b/read (defined at /home/ubuntu/.local/lib/python3.7/site-packages/tensorflow_core/python/framework/ops.py:1748) ]]

Original stack trace for 'pi/linear/b/read':
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/.local/lib/python3.7/site-packages/traitlets/config/application.py", line 845, in launch_instance
    app.start()
  File "/home/ubuntu/.local/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 612, in start
    self.io_loop.start()
  File "/home/ubuntu/.local/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 199, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.7/

In [21]:
pi.X

<tf.Tensor 'Ob:0' shape=(?, 72, 96, 16) dtype=uint8>